In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn.metrics as metrics
import numpy as np

In [2]:
df = pd.read_csv('data/soja_cepea_brasil.csv', delimiter = ';')
df['data'] = pd.to_datetime(df['data'])
df = df.set_index('data')
df = df.sort_index()

In [3]:
df = df.astype(str).stack().str.replace(',','.').unstack().astype(float)
# df = df.drop(['preco_us'], axis = 1)

### Feature Engineering

In [4]:
df2 = df[['preco_br']]

df2.loc[:, 'yesterday'] = df2.loc[:,'preco_br'].shift()
df2.loc[:, 'yesterday_dif'] = df2.loc[:, 'yesterday'].diff()

df2.loc[:, 'yesterday_1'] = df2.loc[:, 'yesterday'].shift()
# df2.loc[:, 'yesterday_1_dif'] = df2.loc[:, 'yesterday_1'].diff()

df2.loc[:, 'last_week'] = df2.loc[:, 'yesterday'].shift(7)
# df2.loc[:, 'last_week_diff'] = df2.loc[:, 'last_week'].diff()


# df2.loc[:, 'last_year'] = df2.loc[:, 'yesterday'].shift(365)
# df2.loc[:, 'last_year_diff'] = df2.loc[:, 'last_year'].diff()
# df2.loc[:, 'last_year_diff'] = df2['yesterday'].sub(df2['last_year'], axis = 0) 

df2.loc[:, 'last_month'] = df2.loc[:, 'yesterday'].shift(30)
# df2.loc[:, 'last_month_diff'] = df2.loc[:, 'last_month'].diff()

df2 = df2.dropna()
df2

,preco_br,yesterday,yesterday_dif,yesterday_1,last_week,last_month
data,,,,,,
1997-08-12,20.59,18.72,-0.74,19.46,18.86,18.10
1997-08-13,18.00,20.59,1.87,18.72,19.67,19.09
1997-08-14,18.07,18.00,-2.59,20.59,18.04,19.24
1997-08-15,18.15,18.07,0.07,18.00,17.97,20.85
1997-08-18,18.17,18.15,0.08,18.07,18.17,19.21
...,...,...,...,...,...,...
2021-10-02,161.66,164.64,0.84,163.80,165.39,141.99
2021-11-01,164.28,161.66,-2.98,164.64,155.72,144.73
2021-11-02,159.73,164.28,2.62,161.66,165.29,144.89


In [5]:
df2.to_csv('data/final/soja_cepea_final.csv')

### Cross-Validation on Time Series 

In [6]:
year_list = df2.index.year.unique().tolist()
splits = {'train': [], 'test': []}

for idx, yr in enumerate(year_list[:-1]):
    train_yr = year_list[:idx+1]
    test_yr = [year_list[idx+1]]
    print('TRAIN: ', train_yr, 'TEST: ',test_yr)
    
    splits['train'].append(df2.loc[df2.index.year.isin(train_yr), :])
    splits['test'].append(df2.loc[df2.index.year.isin(test_yr), :])
    

TRAIN:  [1997] TEST:  [1998]
TRAIN:  [1997, 1998] TEST:  [1999]
TRAIN:  [1997, 1998, 1999] TEST:  [2000]
TRAIN:  [1997, 1998, 1999, 2000] TEST:  [2001]
TRAIN:  [1997, 1998, 1999, 2000, 2001] TEST:  [2002]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002] TEST:  [2003]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003] TEST:  [2004]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004] TEST:  [2005]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005] TEST:  [2006]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006] TEST:  [2007]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007] TEST:  [2008]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008] TEST:  [2009]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009] TEST:  [2010]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010] TEST:  [2011]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 200

In [7]:
model = LinearRegression()

for i, yr in enumerate(year_list[:-1]):
    X_train = splits['train'][i].drop(['preco_br'], axis = 1).values # X
    y_train = splits['train'][i]['preco_br'].values # Y
    
    X_test =  splits['test'][i].drop(['preco_br'], axis = 1).values
    y_test = splits['test'][i]['preco_br'].values
    
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    
    r2 = metrics.r2_score(y_test, pred)
    print(r2)

-1.4435384275823022
0.4419077443227315
0.3031319925054706
0.6813128221228524
0.693457181988826
0.4545714927450224
0.6261830228275518
0.4010801912246956
0.4573520454389284
0.6771997866123034
-0.26446633971364664
0.42250252969114777
0.5849397291865702
0.232709499894724
0.6580825583012272
0.4742442325680092
0.46035148922239144
0.6883096687060504
0.33791641464347233
0.20141562208941322
0.470997731342656
0.6096764937723083
0.6479463260286848
-1.5532475978516422


### Testando a regressao com os dados aleatorizados

In [8]:
X = df2.drop(['preco_br'], axis = 1).values
y = df2.loc[:, 'preco_br']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [10]:
model_v1 = LinearRegression()
model_v1.fit(X_train, y_train)
predict_v1 = model_v1.predict(X_test)
metrics.r2_score(y_test, predict_v1)

0.9678598175674128